# DSML Group Project

import statements

In [5]:
import pandas as pd
import numpy as np
import re
from datetime import datetime #for working with times objects
from datetime import timedelta #for working with times objects

# Task 1: Data Collection and Preparation

In [6]:
# You have been provided with a full dataset of bike sharing
# rentals. Select the cities you have been allocated and clean your dataset for use in later stages of your
# project. Briefly describe how you proceeded and how you dealt with possible missing/erroneous data.

In [7]:
# get data from la 2019


la_set_q1 = pd.read_csv("metro-bike-share-trips-2019-q1.csv", low_memory = False)
la_set_q2 = pd.read_csv("metro-bike-share-trips-2019-q2.csv", low_memory = False)
la_set_q3 = pd.read_csv("metro-bike-share-trips-2019-q3.csv", low_memory = False)
la_set_q4 = pd.read_csv("metro-bike-share-trips-2019-q4.csv", low_memory = False)

la_set = pd.concat([la_set_q1, la_set_q2], ignore_index=True)
la_set = pd.concat([la_set, la_set_q3], ignore_index=True)
la_set = pd.concat([la_set, la_set_q4], ignore_index=True)

#converting start and end time into timestamps

la_set['start_time'] = pd.DatetimeIndex(la_set['start_time'])
la_set['end_time'] = pd.DatetimeIndex(la_set['end_time'])


In [8]:

#adding the exact_duration to the dataframe
la_set["exact_duration"] = la_set["end_time"]-la_set["start_time"]


In [9]:
la_set

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,exact_duration
0,112536773,7,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,34.052872,-118.247490,3051,34.045422,-118.253517,06468,1,One Way,Walk-up,standard,0 days 00:07:00
1,112536772,6,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,34.052872,-118.247490,3051,34.045422,-118.253517,12311,1,One Way,Walk-up,standard,0 days 00:06:00
2,112538689,32,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,34.051941,-118.243530,3075,34.042110,-118.256187,05992,1,One Way,Walk-up,standard,0 days 00:32:00
3,112538688,30,2019-01-01 00:20:00,2019-01-01 00:50:00,3030,34.051941,-118.243530,3075,34.042110,-118.256187,05860,1,One Way,Walk-up,standard,0 days 00:30:00
4,112538687,28,2019-01-01 00:22:00,2019-01-01 00:50:00,3030,34.051941,-118.243530,3075,34.042110,-118.256187,06006,1,One Way,Walk-up,standard,0 days 00:28:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290337,134982544,1440,2019-12-31 23:35:51,2020-01-02 17:13:50,4491,34.047440,-118.247940,4491,34.047440,-118.247940,5903,30,Round Trip,Monthly Pass,standard,1 days 17:37:59
290338,134866394,9,2019-12-31 23:41:52,2019-12-31 23:50:58,4491,34.047440,-118.247940,3022,34.046070,-118.233093,18912,30,One Way,Monthly Pass,electric,0 days 00:09:06
290339,134866292,4,2019-12-31 23:43:19,2019-12-31 23:47:41,3051,34.045422,-118.253517,3064,34.046810,-118.256981,12298,365,One Way,Annual Pass,standard,0 days 00:04:22
290340,134866392,5,2019-12-31 23:48:17,2019-12-31 23:53:55,3064,34.046810,-118.256981,3074,34.044170,-118.261169,19053,365,One Way,Annual Pass,electric,0 days 00:05:38


In [10]:
# remove virtual station VS (ID 3000)
la_set_dropVS = la_set[(la_set["start_station"] != 3000) & (la_set["end_station"] != 3000)]
la_set_dropVS


,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,exact_duration
0,112536773,7,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,34.052872,-118.247490,3051,34.045422,-118.253517,06468,1,One Way,Walk-up,standard,0 days 00:07:00
1,112536772,6,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,34.052872,-118.247490,3051,34.045422,-118.253517,12311,1,One Way,Walk-up,standard,0 days 00:06:00
2,112538689,32,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,34.051941,-118.243530,3075,34.042110,-118.256187,05992,1,One Way,Walk-up,standard,0 days 00:32:00
3,112538688,30,2019-01-01 00:20:00,2019-01-01 00:50:00,3030,34.051941,-118.243530,3075,34.042110,-118.256187,05860,1,One Way,Walk-up,standard,0 days 00:30:00
4,112538687,28,2019-01-01 00:22:00,2019-01-01 00:50:00,3030,34.051941,-118.243530,3075,34.042110,-118.256187,06006,1,One Way,Walk-up,standard,0 days 00:28:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290337,134982544,1440,2019-12-31 23:35:51,2020-01-02 17:13:50,4491,34.047440,-118.247940,4491,34.047440,-118.247940,5903,30,Round Trip,Monthly Pass,standard,1 days 17:37:59
290338,134866394,9,2019-12-31 23:41:52,2019-12-31 23:50:58,4491,34.047440,-118.247940,3022,34.046070,-118.233093,18912,30,One Way,Monthly Pass,electric,0 days 00:09:06
290339,134866292,4,2019-12-31 23:43:19,2019-12-31 23:47:41,3051,34.045422,-118.253517,3064,34.046810,-118.256981,12298,365,One Way,Annual Pass,standard,0 days 00:04:22
290340,134866392,5,2019-12-31 23:48:17,2019-12-31 23:53:55,3064,34.046810,-118.256981,3074,34.044170,-118.261169,19053,365,One Way,Annual Pass,electric,0 days 00:05:38


In [11]:
#how many lines were dropped
lines_dropped = len(la_set)-len(la_set_dropVS)
print("{} lines were dropped.".format(lines_dropped))

4664 lines were dropped.


In [12]:
# start end-stations "Metro Bike Share Free Bikes" ?? station 4285

# drop cells where station out of service OOS

la_set_dropOoS = la_set_dropVS[(la_set_dropVS["start_station"] != 4285)
                               & (la_set_dropVS["end_station"] != 4285)]
la_set_dropOoS


,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,exact_duration
0,112536773,7,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,34.052872,-118.247490,3051,34.045422,-118.253517,06468,1,One Way,Walk-up,standard,0 days 00:07:00
1,112536772,6,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,34.052872,-118.247490,3051,34.045422,-118.253517,12311,1,One Way,Walk-up,standard,0 days 00:06:00
2,112538689,32,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,34.051941,-118.243530,3075,34.042110,-118.256187,05992,1,One Way,Walk-up,standard,0 days 00:32:00
3,112538688,30,2019-01-01 00:20:00,2019-01-01 00:50:00,3030,34.051941,-118.243530,3075,34.042110,-118.256187,05860,1,One Way,Walk-up,standard,0 days 00:30:00
4,112538687,28,2019-01-01 00:22:00,2019-01-01 00:50:00,3030,34.051941,-118.243530,3075,34.042110,-118.256187,06006,1,One Way,Walk-up,standard,0 days 00:28:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290337,134982544,1440,2019-12-31 23:35:51,2020-01-02 17:13:50,4491,34.047440,-118.247940,4491,34.047440,-118.247940,5903,30,Round Trip,Monthly Pass,standard,1 days 17:37:59
290338,134866394,9,2019-12-31 23:41:52,2019-12-31 23:50:58,4491,34.047440,-118.247940,3022,34.046070,-118.233093,18912,30,One Way,Monthly Pass,electric,0 days 00:09:06
290339,134866292,4,2019-12-31 23:43:19,2019-12-31 23:47:41,3051,34.045422,-118.253517,3064,34.046810,-118.256981,12298,365,One Way,Annual Pass,standard,0 days 00:04:22
290340,134866392,5,2019-12-31 23:48:17,2019-12-31 23:53:55,3064,34.046810,-118.256981,3074,34.044170,-118.261169,19053,365,One Way,Annual Pass,electric,0 days 00:05:38


In [13]:
#how many lines were dropped
lines_dropped = len(la_set_dropVS)-len(la_set_dropOoS)
print("{} lines were dropped.".format(lines_dropped))

6207 lines were dropped.


In [14]:
# Trips over 24 hours (long trips)

# drop cells where duration is over 24h

#type(la_set_dropOoS["exact_duration"].loc[0])


la_set_dropLt = la_set_dropOoS[la_set_dropOoS["duration"] < (24*60)]

#more complex but unnessesary
#la_set_dropLt = la_set_dropOoS[la_set_dropOoS["exact_duration"].apply(lambda x: x.total_seconds()) < (24*60*60)]

#[(la_set_dropOoS["exact_duration"].total_seconds())  > (24*60*60)]

la_set_dropLt

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,exact_duration
0,112536773,7,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,34.052872,-118.247490,3051,34.045422,-118.253517,06468,1,One Way,Walk-up,standard,0 days 00:07:00
1,112536772,6,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,34.052872,-118.247490,3051,34.045422,-118.253517,12311,1,One Way,Walk-up,standard,0 days 00:06:00
2,112538689,32,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,34.051941,-118.243530,3075,34.042110,-118.256187,05992,1,One Way,Walk-up,standard,0 days 00:32:00
3,112538688,30,2019-01-01 00:20:00,2019-01-01 00:50:00,3030,34.051941,-118.243530,3075,34.042110,-118.256187,05860,1,One Way,Walk-up,standard,0 days 00:30:00
4,112538687,28,2019-01-01 00:22:00,2019-01-01 00:50:00,3030,34.051941,-118.243530,3075,34.042110,-118.256187,06006,1,One Way,Walk-up,standard,0 days 00:28:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290336,134866192,8,2019-12-31 23:34:46,2019-12-31 23:42:28,3063,34.048038,-118.253738,3069,34.050880,-118.248253,12019,1,One Way,Walk-up,standard,0 days 00:07:42
290338,134866394,9,2019-12-31 23:41:52,2019-12-31 23:50:58,4491,34.047440,-118.247940,3022,34.046070,-118.233093,18912,30,One Way,Monthly Pass,electric,0 days 00:09:06
290339,134866292,4,2019-12-31 23:43:19,2019-12-31 23:47:41,3051,34.045422,-118.253517,3064,34.046810,-118.256981,12298,365,One Way,Annual Pass,standard,0 days 00:04:22
290340,134866392,5,2019-12-31 23:48:17,2019-12-31 23:53:55,3064,34.046810,-118.256981,3074,34.044170,-118.261169,19053,365,One Way,Annual Pass,electric,0 days 00:05:38


In [15]:
#how many lines were dropped
lines_dropped = len(la_set_dropOoS)-len(la_set_dropLt)
print("{} lines were dropped.".format(lines_dropped))

449 lines were dropped.


In [16]:
# same start and  end-stations and duration 5min or below

# drop cells where are short round trips






la_set_dropSrt = la_set_dropLt[((la_set_dropLt["start_station"] == la_set_dropLt["end_station"])
                               & (la_set_dropLt["duration"] > 5))
                               | (la_set_dropLt["start_station"] != la_set_dropLt["end_station"])]
la_set_dropSrt


,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,exact_duration
0,112536773,7,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,34.052872,-118.247490,3051,34.045422,-118.253517,06468,1,One Way,Walk-up,standard,0 days 00:07:00
1,112536772,6,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,34.052872,-118.247490,3051,34.045422,-118.253517,12311,1,One Way,Walk-up,standard,0 days 00:06:00
2,112538689,32,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,34.051941,-118.243530,3075,34.042110,-118.256187,05992,1,One Way,Walk-up,standard,0 days 00:32:00
3,112538688,30,2019-01-01 00:20:00,2019-01-01 00:50:00,3030,34.051941,-118.243530,3075,34.042110,-118.256187,05860,1,One Way,Walk-up,standard,0 days 00:30:00
4,112538687,28,2019-01-01 00:22:00,2019-01-01 00:50:00,3030,34.051941,-118.243530,3075,34.042110,-118.256187,06006,1,One Way,Walk-up,standard,0 days 00:28:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290336,134866192,8,2019-12-31 23:34:46,2019-12-31 23:42:28,3063,34.048038,-118.253738,3069,34.050880,-118.248253,12019,1,One Way,Walk-up,standard,0 days 00:07:42
290338,134866394,9,2019-12-31 23:41:52,2019-12-31 23:50:58,4491,34.047440,-118.247940,3022,34.046070,-118.233093,18912,30,One Way,Monthly Pass,electric,0 days 00:09:06
290339,134866292,4,2019-12-31 23:43:19,2019-12-31 23:47:41,3051,34.045422,-118.253517,3064,34.046810,-118.256981,12298,365,One Way,Annual Pass,standard,0 days 00:04:22
290340,134866392,5,2019-12-31 23:48:17,2019-12-31 23:53:55,3064,34.046810,-118.256981,3074,34.044170,-118.261169,19053,365,One Way,Annual Pass,electric,0 days 00:05:38


In [17]:
#how many lines were dropped
lines_dropped = len(la_set_dropLt)-len(la_set_dropSrt)
print("{} lines were dropped.".format(lines_dropped))

8439 lines were dropped.


In [18]:
# trips under 1min

# drop cells where exact_duration is under 1min

la_set_dropOm = la_set_dropSrt[la_set_dropOoS["exact_duration"].apply(lambda x: x.total_seconds()) > 59]

#how many lines were dropped
lines_dropped = len(la_set_dropSrt)-len(la_set_dropOm)
print("{} lines were dropped.".format(lines_dropped))

20 lines were dropped.


<ipython-input-18-e5def1867b97>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  la_set_dropOm = la_set_dropSrt[la_set_dropOoS["exact_duration"].apply(lambda x: x.total_seconds()) > 59]


In [19]:
#rows with null values

# drop rows which have null values


la_set_dropNv = la_set_dropOm.dropna(axis=0, inplace=False)
la_set_dropNv

,trip_id,duration,start_time,end_time,start_station,start_lat,start_lon,end_station,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,bike_type,exact_duration
0,112536773,7,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,34.052872,-118.247490,3051,34.045422,-118.253517,06468,1,One Way,Walk-up,standard,0 days 00:07:00
1,112536772,6,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,34.052872,-118.247490,3051,34.045422,-118.253517,12311,1,One Way,Walk-up,standard,0 days 00:06:00
2,112538689,32,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,34.051941,-118.243530,3075,34.042110,-118.256187,05992,1,One Way,Walk-up,standard,0 days 00:32:00
3,112538688,30,2019-01-01 00:20:00,2019-01-01 00:50:00,3030,34.051941,-118.243530,3075,34.042110,-118.256187,05860,1,One Way,Walk-up,standard,0 days 00:30:00
4,112538687,28,2019-01-01 00:22:00,2019-01-01 00:50:00,3030,34.051941,-118.243530,3075,34.042110,-118.256187,06006,1,One Way,Walk-up,standard,0 days 00:28:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290336,134866192,8,2019-12-31 23:34:46,2019-12-31 23:42:28,3063,34.048038,-118.253738,3069,34.050880,-118.248253,12019,1,One Way,Walk-up,standard,0 days 00:07:42
290338,134866394,9,2019-12-31 23:41:52,2019-12-31 23:50:58,4491,34.047440,-118.247940,3022,34.046070,-118.233093,18912,30,One Way,Monthly Pass,electric,0 days 00:09:06
290339,134866292,4,2019-12-31 23:43:19,2019-12-31 23:47:41,3051,34.045422,-118.253517,3064,34.046810,-118.256981,12298,365,One Way,Annual Pass,standard,0 days 00:04:22
290340,134866392,5,2019-12-31 23:48:17,2019-12-31 23:53:55,3064,34.046810,-118.256981,3074,34.044170,-118.261169,19053,365,One Way,Annual Pass,electric,0 days 00:05:38


In [20]:
#how many lines were dropped
lines_dropped = len(la_set_dropOm)-len(la_set_dropNv)
print("{} lines were dropped.".format(lines_dropped))

2408 lines were dropped.


In [21]:
la_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290342 entries, 0 to 290341
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype          
---  ------               --------------   -----          
 0   trip_id              290342 non-null  int64          
 1   duration             290342 non-null  int64          
 2   start_time           290342 non-null  datetime64[ns] 
 3   end_time             290342 non-null  datetime64[ns] 
 4   start_station        290342 non-null  int64          
 5   start_lat            285505 non-null  float64        
 6   start_lon            285505 non-null  float64        
 7   end_station          290342 non-null  int64          
 8   end_lat              279757 non-null  float64        
 9   end_lon              279757 non-null  float64        
 10  bike_id              290342 non-null  object         
 11  plan_duration        290342 non-null  int64          
 12  trip_route_category  290342 non-null  object         
 13 

In [22]:
la_set_dropNv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 268155 entries, 0 to 290341
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype          
---  ------               --------------   -----          
 0   trip_id              268155 non-null  int64          
 1   duration             268155 non-null  int64          
 2   start_time           268155 non-null  datetime64[ns] 
 3   end_time             268155 non-null  datetime64[ns] 
 4   start_station        268155 non-null  int64          
 5   start_lat            268155 non-null  float64        
 6   start_lon            268155 non-null  float64        
 7   end_station          268155 non-null  int64          
 8   end_lat              268155 non-null  float64        
 9   end_lon              268155 non-null  float64        
 10  bike_id              268155 non-null  object         
 11  plan_duration        268155 non-null  int64          
 12  trip_route_category  268155 non-null  object         
 13 

In [15]:
# assign la_set to dataset where Out Of Service and Virtual Station are removed
la_set = la_set_dropNv
#reset indices
la_set.reset_index(drop=True)
la_set

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name
0,2019-01-01 00:07:00,2019-01-01 00:14:00,3046,3051,06468,Walk-up,2nd & Hill,7th & Broadway
1,2019-01-01 00:08:00,2019-01-01 00:14:00,3046,3051,12311,Walk-up,2nd & Hill,7th & Broadway
2,2019-01-01 00:18:00,2019-01-01 00:50:00,3030,3075,05992,Walk-up,Main & 1st,Broadway & 9th
3,2019-01-01 00:20:00,2019-01-01 00:50:00,3030,3075,05860,Walk-up,Main & 1st,Broadway & 9th
4,2019-01-01 00:22:00,2019-01-01 00:50:00,3030,3075,06006,Walk-up,Main & 1st,Broadway & 9th
...,...,...,...,...,...,...,...,...
290337,2019-12-31 23:35:51,2020-01-02 17:13:50,4491,4491,5903,Monthly Pass,Main & Winston,Main & Winston
290338,2019-12-31 23:41:52,2019-12-31 23:50:58,4491,3022,18912,Monthly Pass,Main & Winston,3rd & Santa Fe
290339,2019-12-31 23:43:19,2019-12-31 23:47:41,3051,3064,12298,Annual Pass,7th & Broadway,Grand & 8th
290340,2019-12-31 23:48:17,2019-12-31 23:53:55,3064,3074,19053,Annual Pass,Grand & 8th,Hope & Olympic


In [17]:
# get data from metro bike share stations
# https://bikeshare.metro.net/about/data/
#file_path=r"\Users\Jan\DSML\SiliconValley-main (1)\SiliconValley-main"

#stations_set = pd.read_csv("metro-bike-share-stations-2021-04-01.csv")

# delete rows with stations only being active after 2019? predict on region?

In [18]:
stations_set

,Station_ID,Station_Name,Day of Go_live_date,Region,Status
0,3000,Virtual Station,7/7/2016,NaN,Active
1,3005,7th & Flower,7/7/2016,DTLA,Active
2,3006,Olive & 8th,7/7/2016,DTLA,Active
3,3007,5th & Grand,7/7/2016,DTLA,Active
4,3008,Figueroa & 9th,7/7/2016,DTLA,Active
...,...,...,...,...,...
334,4577,Inglewood & Braddock,12/9/2020,Westside,Active
335,4580,Centinela & Lucile,11/27/2020,Westside,Active
336,4581,Playa Vista Library,2/2/2021,Westside,Active
337,4582,Westlawn & Beatrice,11/17/2020,Westside,Active


In [19]:
# Out of Service -> delete rows?
# 2019-02-28 13:35:00,2019-02-28 15:24:00,4344,4286,16004,Walk-up,Downtown Santa Monica Expo Line Station,Metro Bike Share Out of Service Area Smart Bike
# isnull/ data completely missing
# virtual station? Virtual Station" is used by staff to check in or check out a bike remotely for a special event 
# or in a situation in which a bike could not otherwise be checked in or out to a station

# any other outliers/ data that could be wrong or missing?

# Task 2: Descriptive Analytics

In [20]:
# Temporal Demand Patterns and Seasonality: 
# How does fleet usage varies during a day, a week and the year // split day in 24 hours? // weekdays/weekend// monthly differences/ summer/winter etc. //
# ideas: work times (daily), seasons, holiday,special events, weather at this point?

In [23]:

fleet_size = len(la_set["bike_id"].unique())
station_amount = len(la_set["start_station_id"].unique())

print("The la fleet consists of {} bikes in 2019".format(fleet_size), "and there are {} staions.".format(station_amount))

The la fleet consists of 3137 bikes in 2019 and there are 232 staions


In [45]:
# Geographical Demand Patterns:
# which stations are popular, which are not?
popular_start_stations = la_set["start_station_id"].value_counts()
print("popular start stations:\n", popular_start_stations.head(20), "\n")

popular_end_stations = la_set["end_station_id"].value_counts()
print("popular end stations:\n", popular_end_stations.head(20), "\n")

print("unpopular start stations:\n", popular_start_stations[-20:], "\n")

print("unpopular end stations:\n", popular_end_stations[-20:], "\n")
# -> count how many times station is mentioned (seperate start and stop?); maybe select routes that appear often

routes = la_set.groupby(['start_station_id', 'end_station_id']).size().sort_values(ascending=False)
print("popular routes: ")
routes.head(15)
# -> use api to visualize?

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html // how to join on station_id to print names next to it + region

popular start stations:
 3005    13983
3030    13098
3014    10866
3031     8708
3006     6669
4345     6418
3035     6230
3042     5999
3064     5596
3007     5130
3067     5072
3038     4927
3074     4822
3082     4737
4346     4560
3069     4537
3027     4442
3032     4216
3049     4001
3063     3984
Name: start_station_id, dtype: int64 

popular end stations:
 3005    14891
3014    12163
3030    11535
3031     9262
4345     6936
3006     6901
3042     6879
3035     6395
3064     5942
3082     5430
3032     4747
3074     4698
3067     4691
3069     4557
3038     4489
3007     4480
3063     4419
4285     4038
4346     4026
3034     3964
Name: end_station_id, dtype: int64 

unpopular start stations:
 4462    33
4213    31
4493    31
4482    30
4473    29
4457    29
4461    29
4463    28
4484    25
4469    17
4108    14
4439    13
4374    13
4363    13
4395    12
4369    12
4373     3
4362     1
4490     1
4327     1
Name: start_station_id, dtype: int64 

unpopular end stations:
 4462 

start_station_id  end_station_id
3030              3014              5135
3014              3030              4397
4345              4345              3769
4346              4346              2118
4344              4344              1447
3005              3005              1440
3030              3042              1397
3042              3030              1396
3014              3016              1302
4126              4126              1276
4346              4345              1129
3016              3014              1097
3031              3005              1045
4285              4285              1024
3005              3031               995
dtype: int64

In [46]:
# 4126, 4825, 4344, 4346, 4345 often occur as a pair of start and end station
# "Some short round trips or long trips may be the result of system or user error, but have been kept in the dataset for completeness" -> delete rows? keep them? delete if under 10 minutes? etc.

# Key Performance Indicators (3+):
# dashboard for fleet operators
# provide immediate overview of current operations
# and how well the fleet is doing in terms of utilization
# revenue, coverage and/or other business-related aspects
# explain KPIs: why? references
# calculate hourly values for the selectes KPIs + visualize
# Which trends? How to explain them

# Task 3: Predictive Analytics

In [47]:
# forecasting total system-level demand in the next hour
# -> develop practical model that predicts rental demand
# as a function of suitable features available in or derived from the datasets (incl. the weather data)

In [48]:
# feature engineering: Develop a rich set of features that you expect to be correlated with your
# target. In this process you can draw on your domain knowledge and/or conduct additional research
# around the topic of demand prediction in vehicle rental networks. Justify your selection of features.

In [49]:
# Model Building: Select three regression algorithms that are suitable for the prediction task at
# hand. Explain and justify why you selected the three algorithms and describe their respective
# advantages and drawbacks.

In [29]:
# Model Evaluation: How well do the models perform? Evaluate and benchmark your models'
# performance using suitable evaluation metrics. Which model would you select for deployment?

In [30]:
# Outlook: How could the selected model be improved further? Explain some of the improvement
# levers that you might focus on in a follow-up project.